In [10]:
import random
import numpy as np

from RobotEnviroment.init import *
from RobotEnviroment.arenas import RectangularArena
from PointClouds.PushFinder import *
from PathPlanning.scanning import *
from PathPlanning.motion import *

### Initialise Robot and Configuration

In [11]:
ON_REAL = False
INITIAL_OBJ_POS = np.array([-.50, .1, .69])

C = setup_config(INITIAL_OBJ_POS, ON_REAL)
bot = startup_robot(C, ON_REAL)

Define an arena to stop the robot from moving the object too far

In [12]:
TABLE_CENTER = np.array([-.23, -.16, .651])
TABLE_DIMS = np.array([.89, .55])
ROBOT_POS = np.array([-.03, -.24, .651])
ROBOT_RING = .29

arena = RectangularArena(middleP=TABLE_CENTER, width=TABLE_DIMS[0], height=TABLE_DIMS[1], middlePCirc=ROBOT_POS, innerR=ROBOT_RING)
arena.plotArena(C)

### Main pushing loop
- Look towards the predicted object position
- Get the object's point cloud and calculate its normals
- Choose a random pushing point in the point cloud
- Calculate the pushing motion
- Push if the motion is feasible

In [13]:
NUMBER_OF_PUSH_TRIALS = 5
predObjPos = INITIAL_OBJ_POS

for i in range(NUMBER_OF_PUSH_TRIALS):

    print("Starting Trial Number ", i+1)

    print("Looking at Object...")
    lookAtObj(predObjPos, bot, C)
    print("Scanning Object...")
    predObjPos, pointCloud = getScannedObject(bot, C, arena)

    if not len(predObjPos):
        print ("Lost the Object!")
        break

    print("Finding Push Points...")
    push_points  = getPushPoints(pointCloud)

    while True:
        if not len(push_points):
            print("No Viable Push Points!")
            break

        pushP = random.choice(push_points)
        push_points.remove(pushP)

        bot.sync(C, .1)
        print("Calculating Push Waypoints...")
        waypoints = pushPCpoint(pushP[0], pushP[1], predObjPos)

        print("Computing KOMO Problem...")
        path, feasible = computeKomo(C, waypoints)
        if feasible:
            print("Pushing Object...")
            bot.move(path, [4.])
            while bot.getTimeToEnd() > 0:
                bot.sync(C, .1)
            break
        else: print("Point is not Feasible!")

Starting Trial Number  1
Looking at Object...
Scanning Object...
Finding Push Points...
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
Calculating Push Waypoints...
Computing KOMO Problem...
Starting Trial Number  2
Looking at Object...
Scanning Object...
Finding Push Points...
Calculating Push Waypoints...
Computing KOMO Problem...
Starting Trial Number  3
Looking at Object...
Scanning Object...
Finding Push Points...
Calculating Push Waypoints...
Computing KOMO Problem...
Starting Trial Number  4
Looking at Object...
Scanning Object...
Finding Push Points...
Calculating Push Waypoints...
Computing KOMO Problem...
Starting Trial Number  5
Looking at Object...
Scanning Object...
Finding Push Points...
Calculating Push Waypoints...
Computing KOMO Problem...


In [14]:
bot.home(C)

In [15]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
